Apuração da Premiação Tina (Individual)



In [78]:
import pandas as pd
import numpy as np
from pathlib import Path

In [79]:
pd.set_option('display.max_columns', None)

In [80]:
# Ondas Telhanorte e Tumelero cujas lojas serão consideradas para apuração
safra_tume=1 # Onda cujas lojas Tumelero serão consideradas
safra_telha=3 #  Onda cujas lojas Telhanorte serão consideradas

In [81]:
# Dia da apuração
# TODO TROCAR A DATA PARA O ANALISTA DE NEGÓCIOS RODAR NO DIA DA APURAÇÃO
# hoje = pd.to_datetime('today').date()
hoje = pd.to_datetime('2022-05-28')
ANO=str(hoje.year)

In [82]:
# TODO FAZER LOOP PARA RODAR APENAS UMA VEZ 
# SELECIONANDO EMPRESA
ID_UNN=1

dict_EMPRESAS={1:"TELHANORTE", 4:"TUMELERO"}
empresa=dict_EMPRESAS[ID_UNN]
print(ID_UNN)
print(empresa)

1
TELHANORTE


In [83]:
# DIA PADRÃO DE APURAÇÃO
#DIA_APURA=27
DIA_APURA=hoje.day

In [84]:
DIA_APURA

28

In [85]:
MESES= ['jan','fev','mar','abr','mai','jun','jul','ago','set','out','nov','dez']
I_MESES= ['01','02','03','04','05','06','07','08','09','10','11','12']
MES=MESES[hoje.month-1]
I_MES=I_MESES[MESES.index(MES)]
I_MES_ANTERIOR=I_MESES[MESES.index(MES)-1]


In [86]:
I_MES_ANTERIOR

'04'

In [87]:
# Período de Apuração   -   Faltas injustificadas no período
# 27 de Janeiro de 2022
DATA_I = pd.to_datetime('2022-'+I_MES_ANTERIOR+'-'+str(DIA_APURA))  # data inicial AAAA-MM-DD
DATA_F = pd.to_datetime('2022-'+I_MES+'-'+str(DIA_APURA-1))  # data final AAAA-MM-DD

In [88]:
print(DATA_I)
print(DATA_F)

2022-04-28 00:00:00
2022-05-27 00:00:00


In [89]:
# PASTAS 

DIR_BASE='C:/apuracao/'
DIR_INFO=DIR_BASE+'info/'
DIR_TINA_BASES=DIR_INFO
DIR_ENTRA=DIR_BASE+'ENTRADA/'
# PASTA PARA MANTER AS BASES ORIGINAIS
DIR_IN=DIR_BASE+ANO+'/'+MES+'/ORIGINAIS/'
# PASTA COM PLANILHA DE CHECAGEM PARA O ANALISTA DE NEGÓCIOS
DIRETORIO_CHECAGEM=DIR_BASE+ANO+'/'+MES+'/CHECAGEM/'
# PASTA PARA PLANILHAS DAS VERBAS PARA RH 
DIR_RH=DIR_BASE+ANO+"/"+MES+"/"+"para_RH/"



In [90]:
# CRIANDO PASTAS DE SAÍDA SE NÃO EXISTIREM
Path(DIR_IN).mkdir(parents=True, exist_ok=True)
Path(DIRETORIO_CHECAGEM).mkdir(parents=True, exist_ok=True)
Path(DIR_RH).mkdir(parents=True, exist_ok=True)


In [91]:
# Arquivos a serem importados
# Incluídas lojas TUMELERO
F_ALVO = DIR_INFO+"target_salesperson.csv" # 
F_VERBAS = DIR_INFO+"/verbas_descricao.xlsx"

F_LOJA_SAFRA=DIR_INFO+"lojas.csv" # 
#f_loja_safra = "info/lojas_pilotos_safra3.xlsx" # Exemplo de janeiro


In [92]:
# TODO Colocar os valores dos parâmetros em arquivo e importar 
# Parâmetros do Modelo
# Modelo Vendedores
# Modelo Vendedores Comissão
gat_min_comissao = 0.0  # Gatilho minimo comissão
ANG_COEF_COMISSAO = 0.01  # Coeficiente angular = 1%
lin_coef_comissao = 0.0  # Coeficiente Linear = 0.0%

# Modelo Vendedores Comissão Turbinada Decoração
# Como as safras 1, 2 e 3 tem os mesmos valores, mantive só um valor para todas as safras
gat_min_decoracao = 0.0  # Gatilho minimo decoração
ANG_COEF_DECORACAO = 0.02  # Coeficiente angular decoração = 2%
lin_coef_decoracao = 0.0  # Coeficiente Linear decoração = 0.0%

# Modelo Vendedores Comissão turbinada Áreas Técnicas
# Como as safras 1, 2 e 3 tem os mesmos valores, mantive só um valor para todas as safras
gat_min_areastecnicas = 0.0  # Gatilho minimo Áreas Técnicas = 0.0%
ANG_COEF_AREASTECNICAS= 0.005  # Coeficiente angular Áreas Técnicas = 0.5%
lin_coef_areastecnicas = 0.0  # Coeficiente Linear decoração = 0.0%

# Modelo Vendedores Prêmio superação vendas
M1 = 100  # meta1 = 100%
M2 = 150  # meta2 = 150%
M3 = 200  # meta3 = 200%
M4 = 250  # meta4 = 250%
B1 = {1:300, 4:200}  # bonus1 -> ganha se bater a meta1; Telhanorte 1 = R$300,00; Tumelero 4= R$200,00
B2 = {1:600, 4:400}  # bonus2 -> ganha se bater a meta2; Telhanorte 1 = R$600,00; Tumelero 4= R$400,00
B3 = {1:900, 4:600}  # bonus3 -> ganha se bater a meta3; Telhanorte 1 = R$900,00; Tumelero 4= R$600,00
B4 = {1:1200, 4:800}  # bonus4  -> ganha se bater a meta4; Telhanorte 1 = R$1200,00; Tumelero 4= R$800,00

# Modelo Vendedores Prêmio Rentabilidade
GAT_MIN_RENTAB = 100.  # Tem que bater 100% da rentabilidade
ANG_COEF_RENTAB = 0.001  # Coeficiente angular da rentabilidade = 0.1%
LIN_COEF_RENTAB = -0.1  # Coeficiente linear da rentabilidade = -0.1
KICKER = {1:200., 4:100} # Premiação inicial quando a rentabilidade alcança 100%; 
                                   # Telhanorte 1 = R$200,00; Tumelero 4 = R$100,00

In [93]:
# Parâmetros do Modelo
# Modelo Assessores
ASS_A1 = 30000  # Alvo1 = R$30.000,00
ASS_A2 = 50000  # Alvo2 = R$50.000,00
ASS_B1 = 100  # Bônus1 caso bata Alvo1
ASS_B2 = 200  # Bônus2 caso bata Alvo2


In [94]:
# Vitor (Wingg) via e-mail no dia da apuração
E_EXTRATO = DIR_ENTRA+"extract_remuneration_"+ANO+I_MES+str(DIA_APURA)+".xlsx"
F_EXTRATO = DIR_IN+"extract_remuneration_"+ANO+I_MES+str(DIA_APURA)+".xlsx"

# Thayna via sharepoint antes da apuração
# F_FALTAS = DIR_IN+"FALTAS_INJUSTIFICADAS_PREMIACOES TINA_23mar2022_ThaynaCarneiro.xlsx"
E_FALTAS = DIR_ENTRA+"PREMIACOES TINA.xlsx"
F_FALTAS = DIR_IN+"PREMIACOES TINA.xlsx"

In [95]:
print('A planilha de checagem da Apuração da PREMIAÇÃO TINA INDIVIDUAL realizada em '+MES+' está aqui:')
print(DIRETORIO_CHECAGEM)
# ARQUIO DE SAÍDA PARA CHECAGEM DO ANALISTA DE NEGÓCIOS - EM EXCEL
F_SAI_CHECAGEM=DIRETORIO_CHECAGEM + "checagem_apuração_PREMIACAO_INDIVIDUAL_"+MES+".xlsx"
print(F_SAI_CHECAGEM)


A planilha de checagem da Apuração da PREMIAÇÃO TINA INDIVIDUAL realizada em mai está aqui:
C:/apuracao/2022/mai/CHECAGEM/
C:/apuracao/2022/mai/CHECAGEM/checagem_apuração_PREMIACAO_INDIVIDUAL_mai.xlsx


In [96]:
# Sufixos possíveis
sufixo="_sem_considerar_faltas_injustificadas"
#sufixo="_analista"

In [97]:
def nome_verbas(DIR_RH, empresa):
    # Verbas para o RH das premiações Tina Individuais
    F_VERBA_ASS=DIR_RH+"VERBA_4118_SEGMENTO_ASSESSORES_"+empresa+".xlsx"
    F_VERBA_VEN_REN=DIR_RH+"VERBA_4119_RENTABILIDADE_VENDAS_"+empresa+".xlsx"
    F_VERBA_VEN_DEC=DIR_RH+"VERBA_4120_DECORACAO_VENDAS_"+empresa+".xlsx"
    F_VERBA_VEN_SUP=DIR_RH+"VERBA_4121_SUPERACAO_VENDAS_"+empresa+".xlsx"
    F_VERBA_VEN_TEC=DIR_RH+"VERBA_4122_AREAS_TECNICAS_VENDAS_"+empresa+".xlsx"
    # Verbas para o RH das premiações Tina Individuais SEM CONSIDERAR FALTAS INJUSTIFICADAS
    F_VERBA_ASS_SEM=DIR_RH+"VERBA_4118_SEGMENTO_ASSESSORES_"+empresa+sufixo+".xlsx"
    F_VERBA_VEN_REN_SEM=DIR_RH+"VERBA_4119_RENTABILIDADE_VENDAS_"+empresa+sufixo+".xlsx"
    F_VERBA_VEN_DEC_SEM=DIR_RH+"VERBA_4120_DECORACAO_VENDAS_"+empresa+sufixo+".xlsx"
    F_VERBA_VEN_SUP_SEM=DIR_RH+"VERBA_4121_SUPERACAO_VENDAS_"+empresa+sufixo+".xlsx"
    F_VERBA_VEN_TEC_SEM=DIR_RH+"VERBA_4122_AREAS_TECNICAS_VENDAS_"+empresa+sufixo+".xlsx"

    return {'F_VERBA_ASS':F_VERBA_ASS, 
        'F_VERBA_VEN_REN':F_VERBA_VEN_REN, 
        'F_VERBA_VEN_DEC':F_VERBA_VEN_DEC, 
        'F_VERBA_VEN_SUP':F_VERBA_VEN_SUP, 
        'F_VERBA_VEN_TEC':F_VERBA_VEN_TEC,
        'F_VERBA_ASS_SEM':F_VERBA_ASS_SEM,
        'F_VERBA_VEN_REN_SEM':F_VERBA_VEN_REN_SEM, 
        'F_VERBA_VEN_DEC_SEM':F_VERBA_VEN_DEC_SEM, 
        'F_VERBA_VEN_SUP_SEM':F_VERBA_VEN_SUP_SEM, 
        'F_VERBA_VEN_TEC_SEM':F_VERBA_VEN_TEC_SEM}

In [98]:
# Nomes dos arquivos de verbas para RH
verbas_telha=nome_verbas(DIR_RH,dict_EMPRESAS[1])
verbas_tume=nome_verbas(DIR_RH,dict_EMPRESAS[4])

In [99]:
verbas_telha['F_VERBA_ASS_SEM']

'C:/apuracao/2022/mai/para_RH/VERBA_4118_SEGMENTO_ASSESSORES_TELHANORTE_sem_considerar_faltas_injustificadas.xlsx'

In [100]:
import shutil
# Fazendo backup das bases nas pasta ORIGINAIS respectiva ao mes e ano da apuração 
shutil.copy2(E_EXTRATO,F_EXTRATO)
shutil.copy2(E_FALTAS,F_FALTAS)

'C:/apuracao/2022/mai/ORIGINAIS/PREMIACOES TINA.xlsx'

In [101]:
nome_lojas[nome_lojas[
    (nome_lojas['id_unn'] == 1) & (nome_lojas['safra'] > safra_telha)
    ]]

,id_unn,cod_loja,sigla_loja,safra,loja,Unnamed: 5
0,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN
6,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,NaN,NaN,NaN,NaN,NaN
8,NaN,NaN,NaN,NaN,NaN,NaN
9,NaN,NaN,NaN,NaN,NaN,NaN


In [102]:
# Carregando lojas TINA da Telhanorte e da Tumelero 
#
nome_lojas = pd.read_csv(F_LOJA_SAFRA, encoding="ISO-8859-1", delimiter=";")
# Deletando lojas que não estão na safra TINA
nome_lojas.drop(nome_lojas[(
    (nome_lojas['id_unn']==1) & (nome_lojas['safra']>safra_telha)
    ) | (
    (nome_lojas['id_unn']==4) & (nome_lojas['safra']>safra_tume)   
    )
    ].index, inplace=True)

In [103]:
# Função para importar comissões separamente por empresa (vendas da ADP)
def importa_comissoes(arquivo):
    # Atribuindo nomes às colunas das comissões
    column_names=[
    'N1',	
    'N2',	
    'N3',	
    'MATRICULA',	
    'POS_NEG',	 
    'VALOR',
    'NONE'
    ]
    print(arquivo)
    print("Importando as comissões")
    print("Arquivo: "+arquivo)
    comissao = pd.read_csv(arquivo, delimiter=";", names=column_names, header=None, skiprows=1)
    # Trocando "," por "." para poder calcular os números no Python
    comissao["VALOR"]=comissao["VALOR"].str.replace(",",".").astype(float)
    ##  Vendas devolvidas como negativas
    print("Valores negativos para vendas devolvidas")
    comissao["VALOR_COR"]=np.where(comissao["POS_NEG"]==3771, -comissao["VALOR"],comissao["VALOR"])
    # Só matrícula e valor das vendas correspondente a essa matrícula
    comissao=comissao[['MATRICULA','VALOR_COR']]
    # Somando as vendas totais de cada matrícula
    n_vendas = comissao.groupby(['MATRICULA']).size().reset_index()
    n_vendas.rename(columns={0:'NUMERO_DE_LOJAS_COM_VENDA'}, inplace=True)
    print("Somando todas as vendas de uma matrícula:")
    comissao = comissao.groupby(['MATRICULA'])['VALOR_COR'].sum().reset_index()
    comissao.rename(columns={'VALOR_COR':'VENDAS_TOT'}, inplace=True)
    comissao['COMISSAO']=comissao['VENDAS_TOT']*0.01
    comissao=pd.merge(comissao,n_vendas, how='left', on=['MATRICULA'] )
    return comissao

In [104]:
# Comissão TELHANORTE
# Ricardo Valdir via e-mail para o Junior no dia da apuração
E_COMISSAO = DIR_ENTRA+"LANC_COMISSAO_"+str(I_MES)+ANO+'_'+dict_EMPRESAS[1].lower() # base de entrada Comissao
F_COMISSAO = DIR_IN+"LANC_COMISSAO_"+str(I_MES)+ANO+'_'+dict_EMPRESAS[1].lower() # cópia da base Comissão a ser mantida
# backup da base de comissão TELHANORTE
shutil.copy2(E_COMISSAO,F_COMISSAO)
# importando a comissão Telhanorte
comissao_telha=importa_comissoes(F_COMISSAO)


C:/apuracao/2022/mai/ORIGINAIS/LANC_COMISSAO_052022_telhanorte
Importando as comissões
Arquivo: C:/apuracao/2022/mai/ORIGINAIS/LANC_COMISSAO_052022_telhanorte
Valores negativos para vendas devolvidas
Somando todas as vendas de uma matrícula:


In [105]:
# Para visualizar as vendas totais e a comissão de um colaborador 
comissao_telha[comissao_telha['MATRICULA'] == 25724]

,MATRICULA,VENDAS_TOT,COMISSAO,NUMERO_DE_LOJAS_COM_VENDA
435,25724,79905.1,799.051,3


In [106]:
# Comissão TUMELERO
# Ricardo Valdir via e-mail para o Junior no dia da apuração
E_COMISSAO = DIR_ENTRA+"LANC_COMISSAO_"+str(I_MES)+ANO+'_'+dict_EMPRESAS[4].lower() # base de entrada Comissao
F_COMISSAO = DIR_IN+"LANC_COMISSAO_"+str(I_MES)+ANO+'_'+dict_EMPRESAS[4].lower() # cópia da base Comissão a ser mantida
# backup da base de comissão TUMELERO
shutil.copy2(E_COMISSAO,F_COMISSAO)
# importando a comissão Tumelero
comissao_tume=importa_comissoes(F_COMISSAO)


C:/apuracao/2022/mai/ORIGINAIS/LANC_COMISSAO_052022_tumelero
Importando as comissões
Arquivo: C:/apuracao/2022/mai/ORIGINAIS/LANC_COMISSAO_052022_tumelero
Valores negativos para vendas devolvidas
Somando todas as vendas de uma matrícula:


In [107]:
# Para visualizar as vendas totais e a comissão de um colaborador 
comissao_tume[comissao_tume['MATRICULA'] == 2359]

,MATRICULA,VENDAS_TOT,COMISSAO,NUMERO_DE_LOJAS_COM_VENDA


In [108]:
# juntando comissão da Telhanorte e Tumelero
comissao_telha['ID_UNN']=1 # Identidade da empresa Telhanorte = 1
comissao_tume['ID_UNN']=4 # Identidade da empresa Tumelero = 4
comissao=pd.concat([
    comissao_telha[['ID_UNN', 'MATRICULA', 'VENDAS_TOT', 'COMISSAO', 'NUMERO_DE_LOJAS_COM_VENDA']],
    comissao_tume[['ID_UNN', 'MATRICULA', 'VENDAS_TOT', 'COMISSAO', 'NUMERO_DE_LOJAS_COM_VENDA']]
    ])



In [109]:
# Lendo os alvos que os vendedores devem atingir por loja
print("Lendo alvos das lojas no arquivo:")
print(F_ALVO)
alvos = pd.read_csv(F_ALVO, delimiter=";", header=0, skiprows=0)
# Renomeando e transformando em número
alvos.rename(columns={'target_sales_salesperson':'Alvo1'}, inplace=True)
alvos['Alvo1']=alvos['Alvo1'].str.replace(",",".").astype(float)
#Transformando floor remuneration em Mínimo Garantido e em número
alvos['Minimo Garantido']=alvos['floor_remuneration'].str.replace(",",".").astype(float)

Lendo alvos das lojas no arquivo:
C:/apuracao/info/target_salesperson.csv


In [110]:
alvos.loc[[0,39]]

,id_unn,cod_loja,Alvo1,target_margin_salesperson_E1,target_margin_salesperson_E2,target_margin_salesperson_E3,target_sales_assistant,target_assistant_E1,target_assistant_E2,target_assistant_E3,floor_remuneration,base_remuneration,Minimo Garantido
0,1,75,160000.0,0,0,0,0,0,0,0,"1283,7",0,1283.7
39,4,510019,195000.0,0,0,0,0,0,0,0,"1653,0","1171,2",1653.0


In [111]:
# Mantendo somente alvos Telhanorte
#alvos=alvos[alvos['id_unn']==1] # id_unn = (1=Telhanorte; 2=Tumelero)
# Mantendo somente alguns parâmetros
alvos= alvos[['cod_loja','Alvo1', 'Minimo Garantido']]

alvos=pd.merge(alvos,nome_lojas[['cod_loja','sigla_loja']],how='left',on='cod_loja')

In [112]:
# TODO Deletar célula se ok
# Dicionário de alvo por codigo da loja
#dict_alvo=dict(zip(alvos['cod_loja'], alvos['Alvo1']))

In [113]:
# TODO Deletar célula se ok
# Dicionário de alvo por codigo da loja
#dict_minimo_garantido=dict(zip(alvos['cod_loja'], alvos['Minimo Garantido']))

In [114]:
# Lendo os alvos que os vendedores devem atingir por loja
#print("Lendo alvos das lojas no arquivo:")
# print(F_ALVO)
# alvos = pd.read_excel(F_ALVO, header=0, skiprows=1)
# alvos['Minimo Garantido']=alvos['Minimo Garantido']/100.
#a[a['Loja']==7992]
# alvos.columns

In [115]:
# Lendo faltas injustificadas
print("Lendo faltas injustificadas no arquivo:")
print(F_FALTAS)
faltas_inj = pd.read_excel(F_FALTAS, header=0)

Lendo faltas injustificadas no arquivo:
C:/apuracao/2022/mai/ORIGINAIS/PREMIACOES TINA.xlsx


In [116]:
faltas = faltas_inj[(faltas_inj['DATA DA OCORRÊNCIA'] >= DATA_I) &
           (faltas_inj['DATA DA OCORRÊNCIA'] <= DATA_F)]
faltas['DATA DA OCORRÊNCIA'].sort_values()

10    2022-04-28
83    2022-04-28
121   2022-04-28
133   2022-04-28
36    2022-04-28
         ...    
307   2022-05-26
329   2022-05-27
308   2022-05-27
340   2022-05-27
322   2022-05-27
Name: DATA DA OCORRÊNCIA, Length: 225, dtype: datetime64[ns]

In [117]:
faltas=pd.merge(
    faltas.rename(columns={
        'EMPRESA':'ID_UNN',
        'NOME ESTAB':'SIGLA_LOJA'
        }),
    nome_lojas[['id_unn','cod_loja','sigla_loja']].rename(columns={
        'id_unn':'ID_UNN',
        'cod_loja':'COD_LOJA',
        'sigla_loja': 'SIGLA_LOJA'
        }),
    how='left',
    on=['ID_UNN', 'SIGLA_LOJA']
    )

In [118]:
faltas.head(5)

,PERIODO - DTINI,PERIODO - DTFIM,ID_UNN,DATA DA OCORRÊNCIA,COD ESTAB,SIGLA_LOJA,COD CR,NOME CR,CÓDIGO DO CENTRO CUSTO CONTABILIDADE,MATR CHEFIA,NOME CHEFIA,MATR EMPR,NOME EMPPR,FUNCAO-DENOMINACAO,COD VERBA,DESCRICAO VERBA,VALOR,COD_LOJA
0,2022-04-16,2022-05-15,1,2022-04-28,32,PAM,344402,PAM - OPERACOES LOJA,344402,11014,REGINALDO DIAS DO NASCIMENTO,20159,ADAILZA SANTOS DA SILVA,Lider de Caixa,9005,Falta Injustificada,7.33,34.0
1,2022-04-16,2022-05-15,1,2022-04-30,32,PAM,344402,PAM - OPERACOES LOJA,344402,11014,REGINALDO DIAS DO NASCIMENTO,20159,ADAILZA SANTOS DA SILVA,Lider de Caixa,9005,Falta Injustificada,7.33,34.0
2,2022-04-16,2022-05-15,1,2022-05-02,32,PAM,344402,PAM - OPERACOES LOJA,344402,11014,REGINALDO DIAS DO NASCIMENTO,20159,ADAILZA SANTOS DA SILVA,Lider de Caixa,9005,Falta Injustificada,7.33,34.0
3,2022-04-16,2022-05-15,1,2022-05-03,32,PAM,344402,PAM - OPERACOES LOJA,344402,11014,REGINALDO DIAS DO NASCIMENTO,20159,ADAILZA SANTOS DA SILVA,Lider de Caixa,9005,Falta Injustificada,7.33,34.0
4,2022-04-16,2022-05-15,1,2022-05-04,32,PAM,344402,PAM - OPERACOES LOJA,344402,11014,REGINALDO DIAS DO NASCIMENTO,20159,ADAILZA SANTOS DA SILVA,Lider de Caixa,9005,Falta Injustificada,7.33,34.0


In [119]:
# Loading Remuneration extract from Tina
print("Lendo Extrato de Remuneração da Tina")
print(F_EXTRATO)
extrato = pd.read_excel(F_EXTRATO)

Lendo Extrato de Remuneração da Tina
C:/apuracao/2022/mai/ORIGINAIS/extract_remuneration_20220528.xlsx


In [120]:
# Somente colaboradores Tina das lojas da safra
print("Considerando somente colaboradores das lojas participantes da Tina")
COLS_TO_KEEP=[
    'store_sk', 
    'registration_id', 
    'segment', 
    'job_simplified',
    'E1_gross_sales', 
    'E2_gross_sales', 
    'E3_gross_sales', 
    'gross_sales', 
    'sales_segment', 
    'rentability'
    ]
so_safra = extrato[extrato["store_sk"].isin(nome_lojas['cod_loja'])][COLS_TO_KEEP].copy(deep=True)
so_safra.columns

Considerando somente colaboradores das lojas participantes da Tina


Index(['store_sk', 'registration_id', 'segment', 'job_simplified',
       'E1_gross_sales', 'E2_gross_sales', 'E3_gross_sales', 'gross_sales',
       'sales_segment', 'rentability'],
      dtype='object')

In [121]:
faltas.columns

Index(['PERIODO - DTINI', 'PERIODO - DTFIM', 'ID_UNN', 'DATA DA OCORRÊNCIA',
       'COD ESTAB', 'SIGLA_LOJA', 'COD CR', 'NOME CR',
       'CÓDIGO DO CENTRO CUSTO CONTABILIDADE', 'MATR CHEFIA', 'NOME CHEFIA',
       'MATR EMPR', 'NOME EMPPR', 'FUNCAO-DENOMINACAO', 'COD VERBA',
       'DESCRICAO VERBA', 'VALOR', 'COD_LOJA'],
      dtype='object')

In [122]:
#para planilha checagem
tina=so_safra.copy(deep=True)


In [123]:
# Verificando quem tem falta injustificada
tina['FALTA']=np.where(
    (tina['store_sk'].isin(faltas['COD_LOJA'])==True) &
    (tina['registration_id'].isin(faltas['MATR EMPR'])==True) 
    ,'SIM','NAO')

In [124]:
tina.head()

,store_sk,registration_id,segment,job_simplified,E1_gross_sales,E2_gross_sales,E3_gross_sales,gross_sales,sales_segment,rentability,FALTA
0,1,1890,E3,salesperson,68514.788,7084.583,343005.411,418604.782,343005.411,-345.958834,NAO
1,1,3402,E3,salesperson,41451.460,15584.358,72147.997,129183.815,72147.997,99.930325,NAO
2,1,7705,E1,salesperson,40476.043,9269.230,83217.959,132963.232,40476.043,103.444778,NAO
3,1,7788,E1,salesperson,76068.116,15115.092,129328.348,220511.556,76068.116,-69.806252,NAO
4,1,10522,E3,salesperson,40902.166,4571.030,482674.918,530005.184,482674.918,100.506720,NAO


In [125]:
# Juntando nome de sigla das lojas
tina=pd.merge(
    tina.rename(columns={'store_sk':'cod_loja'}),
    nome_lojas[['cod_loja', 'sigla_loja']],
    how='left',on='cod_loja'
    )

In [126]:
# Renomeando matricula para juntar com a comissão
tina.rename(columns={'registration_id':'MATRICULA'}, inplace=True)


In [127]:
# Identificando lojas da empresa (ID_UNN) Telhanorte = 1 e Tumelero = 4
tina['ID_UNN']=np.where(tina['cod_loja']<510000,1,4)

In [128]:
# # Juntando com a comissão
tina=tina.merge(
    comissao,
    how='left',on=['ID_UNN','MATRICULA'])

In [129]:
tina.sample(5)

,cod_loja,MATRICULA,segment,job_simplified,E1_gross_sales,E2_gross_sales,E3_gross_sales,gross_sales,sales_segment,rentability,FALTA,sigla_loja,ID_UNN,VENDAS_TOT,COMISSAO,NUMERO_DE_LOJAS_COM_VENDA
110,3,28488,E3,salesperson,25051.836,6066.059,266740.407,297858.302,266740.407,-309.183665,NAO,MGN,1,297858.30,2978.5830,1.0
169,9,29630,E1,assistant,32419.161,18987.432,45917.512,97324.105,32419.161,-2.193520,NAO,SA1,1,97324.11,973.2411,2.0
565,36,20626,E1,salesperson,35356.174,8472.460,103538.566,147367.200,35356.174,100.393620,NAO,CTG,1,147367.20,1473.6720,1.0
46,3,10880,E3,salesperson,19374.278,6790.704,267379.807,293544.789,267379.807,102.689357,NAO,MGN,1,293544.79,2935.4479,1.0
720,54,27056,E1,salesperson,126112.541,6973.887,149373.533,284755.291,126112.541,-691.812252,NAO,MGA,1,284755.29,2847.5529,1.0


In [130]:

tina=pd.merge(
    tina,
    alvos[['cod_loja','Alvo1', 'Minimo Garantido']],
    how='left', on='cod_loja'
    )

In [131]:
tina.sample(5)

,cod_loja,MATRICULA,segment,job_simplified,E1_gross_sales,E2_gross_sales,E3_gross_sales,gross_sales,sales_segment,rentability,FALTA,sigla_loja,ID_UNN,VENDAS_TOT,COMISSAO,NUMERO_DE_LOJAS_COM_VENDA,Alvo1,Minimo Garantido
920,510019,10821,E3,salesperson,12928.120,652.000,15249.650,28829.770,15249.650,100.792817,NAO,MTZ,4,30883.80,308.8380,1.0,195000.00,1653.0
126,8,16852,E3,salesperson,19836.202,12094.807,67199.228,99130.237,67199.228,103.949769,NAO,PGD,1,99130.24,991.3024,1.0,230402.88,2133.4
845,71,27542,E1,assistant,9269.460,946.680,7896.270,18112.410,9269.460,97.884090,NAO,RDV,1,18112.41,181.1241,1.0,245148.00,2042.9
948,510078,10346,E3,salesperson,23266.890,9953.140,75057.790,108277.820,75057.790,102.697677,NAO,NHA,4,110149.18,1101.4918,1.0,173500.00,1656.0
564,36,19497,E3,salesperson,25376.738,13888.699,138137.475,177402.912,138137.475,-39.459289,NAO,CTG,1,177402.91,1774.0291,1.0,205000.00,1291.4


In [132]:
# Eliminando alvos de assessores
#tina['Alvo1']=np.where(tina['job_simplified']=='assistant',np.nan,tina['Alvo1'])
#tina['VENDAS_SOBRE_ALVO']=tina['VENDAS_TOT']/tina['Alvo1']

tina.loc[tina.job_simplified=='assistant', 'Alvo1'] = None
tina['VENDAS_SOBRE_ALVO'] =  None
tina.loc[tina.job_simplified=='salesperson', 'VENDAS_SOBRE_ALVO'] = tina['VENDAS_TOT']/tina['Alvo1']
tina.sample()
#tina.loc[
#    (tina.ID_UNN == 1) &  # ID_UNN para Telhanorte
#    (tina.job_simplified=='salesperson'), 
#    'PREMIO_RENTABILIDADE'
#    ] = tina.apply(
#        lambda x: alvo_bonus_rentabilidade(
#            1,  # ID_UNN para Telhanorte
#            x['VENDAS_TOT'], x['rentability'], x['sales_segment'], x['Alvo1']
#            ), axis=1)

,cod_loja,MATRICULA,segment,job_simplified,E1_gross_sales,E2_gross_sales,E3_gross_sales,gross_sales,sales_segment,rentability,FALTA,sigla_loja,ID_UNN,VENDAS_TOT,COMISSAO,NUMERO_DE_LOJAS_COM_VENDA,Alvo1,Minimo Garantido,VENDAS_SOBRE_ALVO
918,510019,10499,E1,salesperson,41271.4,32808.82,117059.68,191139.9,41271.4,102.872283,NAO,MTZ,4,194011.35,1940.1135,1.0,195000.0,1653.0,0.99493


In [133]:
tina[tina['job_simplified']=='assistant']

,cod_loja,MATRICULA,segment,job_simplified,E1_gross_sales,E2_gross_sales,E3_gross_sales,gross_sales,sales_segment,rentability,FALTA,sigla_loja,ID_UNN,VENDAS_TOT,COMISSAO,NUMERO_DE_LOJAS_COM_VENDA,Alvo1,Minimo Garantido,VENDAS_SOBRE_ALVO
8,1,25178,E2,assistant,17609.381,3417.524,9070.178,30097.083,3417.524,-897.788765,NAO,ANH,1,30097.08,300.9708,1.0,NaN,1866.0,None
9,1,25345,E1,assistant,8095.201,2760.429,17715.689,28571.319,8095.201,80.565628,NAO,ANH,1,28571.32,285.7132,1.0,NaN,1866.0,None
12,1,27242,E1,assistant,51384.227,7904.467,32788.917,92077.611,51384.227,-190.570906,NAO,ANH,1,92077.61,920.7761,1.0,NaN,1866.0,None
13,1,27359,E1,assistant,43644.627,7356.856,49167.624,100169.107,43644.627,-352.253926,NAO,ANH,1,97310.11,973.1011,1.0,NaN,1866.0,None
14,1,27368,E1,assistant,7238.272,4380.548,13811.820,25430.640,7238.272,96.359786,NAO,ANH,1,25430.64,254.3064,1.0,NaN,1866.0,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
895,75,25932,E1,assistant,2964.829,2008.420,4960.070,9933.319,2964.829,103.930166,SIM,AMZ,1,9933.32,99.3332,1.0,NaN,1283.7,None
901,75,27700,E2,assistant,-21.950,NaN,NaN,-21.950,0.000,97.972411,SIM,AMZ,1,-21.95,-0.2195,1.0,NaN,1283.7,None
902,75,27880,E1,assistant,2901.970,3261.980,4485.130,10649.080,2901.970,104.364759,NAO,AMZ,1,10649.08,106.4908,1.0,NaN,1283.7,None
903,75,28273,E1,assistant,16241.799,8528.591,13820.189,38590.579,16241.799,-110.541333,NAO,AMZ,1,38590.58,385.9058,1.0,NaN,1283.7,None


In [134]:
# Eliminando minimos garantidos de assessores
tina['Minimo Garantido']=np.where(tina['job_simplified']=='assistant',np.nan,tina['Minimo Garantido'])

In [135]:
# TODO DELETAR SE NÃO DER ERRO
# Eliminando vendas sobre alvo de assessores
# tina['VENDAS_SOBRE_ALVO']=np.where(tina['job_simplified']=='assistant',np.nan,tina['VENDAS_SOBRE_ALVO'])

In [136]:
# Vendedores atingiu alvo ou não atingiu alvo
tina.loc[
    (tina.job_simplified=='salesperson') &
    (tina.VENDAS_SOBRE_ALVO>=1.), 'ATINGIU_ALVO'] = 'SIM'
tina.loc[
    (tina.job_simplified=='salesperson') &
    (tina.VENDAS_SOBRE_ALVO<1.), 'ATINGIU_ALVO'] = 'NAO'

# TODO DELETAR LINHA SE 
# tina['ATINGIU_ALVO']=np.where(tina['VENDAS_SOBRE_ALVO']>=1.,'SIM','NAO')

In [137]:
# Assessores atingiu alvo
tina.loc[
    (tina.job_simplified=='assistant') &
    (tina.sales_segment>=30000), 'ATINGIU_ALVO'] = '30mil'
tina.loc[
    (tina.job_simplified=='assistant') &
    (tina.sales_segment>=50000), 'ATINGIU_ALVO'] = '50mil'
tina.loc[
    (tina.job_simplified=='assistant') &
    (tina.sales_segment<30000), 'ATINGIU_ALVO'] = 'NAO'


In [138]:
# RECEBE PREMIAÇÃO TINA SE ATINGIU ALVO E NÃO TEM FALTA INJUSTIFICADA
tina['RECEBE_PREMIACAO']=np.where((
    (tina['ATINGIU_ALVO']=='SIM') |
    (tina['ATINGIU_ALVO']=='50mil') |
    (tina['ATINGIU_ALVO']=='30mil')) & 
    (tina['FALTA']=='NAO'),
     'SIM', 'NAO')


In [169]:
tina[['MATRICULA','VENDAS_TOT', 'ATINGIU_ALVO','RECEBE_PREMIACAO']][tina['RECEBE_PREMIACAO']!='NAO']
#tina[['MATRICULA','VENDAS_TOT', 'FALTA', 'ATINGIU_ALVO','RECEBE_PREMIACAO']][tina['ATINGIU_ALVO']!='NAO'].sample(15)
#tina[['MATRICULA','VENDAS_TOT', 'FALTA', 'ATINGIU_ALVO','RECEBE_PREMIACAO']][tina['FALTA']=='SIM'].sample(15)

,MATRICULA,VENDAS_TOT,ATINGIU_ALVO,RECEBE_PREMIACAO
0,1890,418604.78,SIM,SIM
4,10522,530005.18,SIM,SIM
5,13170,381787.63,SIM,SIM
12,27242,92077.61,50mil,SIM
13,27359,97310.11,30mil,SIM
...,...,...,...,...
1008,10151,178678.13,SIM,SIM
1011,11949,202854.24,SIM,SIM
1017,12779,193072.83,SIM,SIM
1023,11492,191129.26,SIM,SIM


In [140]:
#Salvando verba para TELHANORTE ou TUMELERO
# Entradas: v verba, n nome_verba, x numero_verba, f f_nome
def salva_verba(empresa,verba,numero_verba,arquivo):
#    v.rename(columns={ v.columns[0]: 'MATRICULA'}, inplace=True)
    verba.rename(columns={ verba.columns[1]: "VALOR" }, inplace=True)
    verba['ZERO'] = 0
    verba['VERBA'] = numero_verba
    if empresa == 'TELHANORTE':
        verba=verba[['MATRICULA', 'ZERO', 'VERBA','VALOR']]
    elif empresa == 'TUMELERO':
         verba=verba[['MATRICULA', 'VERBA', 'ZERO','VALOR']]
    else: 
        print('Erro!!! Não veio a empresa nem como TELHANORTE e nem como TUMELERO')
        return
    verba = verba[verba['VALOR'] > 0]
    # Saving as a excel file for RH
    verba.to_excel(arquivo, sheet_name="VERBA "+str(numero_verba), index=False)
    

In [141]:
tina[tina['VENDAS_SOBRE_ALVO']>1.]

,cod_loja,MATRICULA,segment,job_simplified,E1_gross_sales,E2_gross_sales,E3_gross_sales,gross_sales,sales_segment,rentability,FALTA,sigla_loja,ID_UNN,VENDAS_TOT,COMISSAO,NUMERO_DE_LOJAS_COM_VENDA,Alvo1,Minimo Garantido,VENDAS_SOBRE_ALVO,ATINGIU_ALVO,RECEBE_PREMIACAO
0,1,1890,E3,salesperson,68514.788,7084.583,343005.411,418604.782,343005.411,-345.958834,NAO,ANH,1,418604.78,4186.0478,1.0,268704.0,1866.0,1.557866,SIM,SIM
4,1,10522,E3,salesperson,40902.166,4571.030,482674.918,530005.184,482674.918,100.506720,NAO,ANH,1,530005.18,5300.0518,1.0,268704.0,1866.0,1.97245,SIM,SIM
5,1,13170,E3,salesperson,88603.133,9693.355,282418.837,381787.635,282418.837,101.249834,NAO,ANH,1,381787.63,3817.8763,1.0,268704.0,1866.0,1.420848,SIM,SIM
26,3,621,E3,salesperson,42178.957,2590.231,395692.422,440461.610,395692.422,100.017962,NAO,MGN,1,440461.61,4404.6161,1.0,280000.0,1985.0,1.573077,SIM,SIM
27,3,2120,E3,salesperson,22019.789,3600.099,346025.201,371645.089,346025.201,104.958829,NAO,MGN,1,371645.09,3716.4509,1.0,280000.0,1985.0,1.327304,SIM,SIM
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1008,510574,10151,E1,salesperson,55718.060,21608.980,96266.740,173593.780,55718.060,102.133868,NAO,IPI,4,178678.13,1786.7813,1.0,173500.0,1653.0,1.029845,SIM,SIM
1011,510574,11949,E1,salesperson,52844.660,12373.540,120998.010,186216.210,52844.660,101.192440,NAO,IPI,4,202854.24,2028.5424,1.0,173500.0,1653.0,1.169189,SIM,SIM
1017,510574,12779,E1,salesperson,49969.770,14951.120,119654.830,184575.720,49969.770,101.267254,NAO,IPI,4,193072.83,1930.7283,1.0,173500.0,1653.0,1.112812,SIM,SIM
1023,510639,11492,E3,salesperson,76015.910,9725.720,86013.400,171755.030,86013.400,102.127720,NAO,CCH,4,191129.26,1911.2926,1.0,173500.0,1697.3,1.10161,SIM,SIM


In [142]:
tina.columns

Index(['cod_loja', 'MATRICULA', 'segment', 'job_simplified', 'E1_gross_sales',
       'E2_gross_sales', 'E3_gross_sales', 'gross_sales', 'sales_segment',
       'rentability', 'FALTA', 'sigla_loja', 'ID_UNN', 'VENDAS_TOT',
       'COMISSAO', 'NUMERO_DE_LOJAS_COM_VENDA', 'Alvo1', 'Minimo Garantido',
       'VENDAS_SOBRE_ALVO', 'ATINGIU_ALVO', 'RECEBE_PREMIACAO'],
      dtype='object')

In [143]:
# Reorganizando a ordem dos parâmtros
tina = tina[['ID_UNN', 'cod_loja', 'sigla_loja', 'MATRICULA', 'segment', 'job_simplified', 'E1_gross_sales',
       'E2_gross_sales', 'E3_gross_sales', 'gross_sales', 'sales_segment',
       'rentability', 'VENDAS_TOT',
       'COMISSAO', 'NUMERO_DE_LOJAS_COM_VENDA', 'Alvo1', 'Minimo Garantido',
       'FALTA', 'VENDAS_SOBRE_ALVO', 'ATINGIU_ALVO', 'RECEBE_PREMIACAO']]

In [144]:
def alvo_bonus_rentabilidade(id_unn, vendas_tot,rentabilidade,vendas_seg,alvo):
#     # Modelo Vendedores Prêmio Rentabilidade
#    T E S T A D O     E     C H E C A D O
# gat_min_rentab = 100.  # Tem que bater 100% da rentabilidade
# ang_coef_rentab = 0.001  # Coeficiente angular da rentabilidade = 0.1%
# lin_coef_rentab = -0.1  # Coeficiente linear da rentabilidade = -0.1
# kicker = 200.
    global GAT_MIN_RENTAB, ANG_COEF_RENTAB, LIN_COEF_RENTAB, KICKER
    if vendas_tot>=alvo and rentabilidade >= GAT_MIN_RENTAB:
    #    # TODO DELETAR SE OK
#        val_rent = KICKER DELETAR SE OK
        # TODO DELETAR SE OK
        #if (rentabilidade-GAT_MIN_RENTAB)>0: DELETAR SE OK
        val_rent=(rentabilidade*ANG_COEF_RENTAB+LIN_COEF_RENTAB)*vendas_seg+KICKER[id_unn]
            # val_rent=val_rent+int(r-gat_min_rentab)*ang_coef_rentab*s
    else:
        val_rent=0.0
    return val_rent



In [145]:
# Calculando Prêmio Rentabilidade para TELHANORTE
tina.loc[
    (tina.ID_UNN == 1) &  # ID_UNN para Telhanorte
    (tina.job_simplified=='salesperson'), 
    'PREMIO_RENTABILIDADE'
    ] = tina.apply(
        lambda x: alvo_bonus_rentabilidade(
            1,  # ID_UNN para Telhanorte
            x['VENDAS_TOT'], x['rentability'], x['sales_segment'], x['Alvo1']
            ), axis=1).copy(deep=True)

In [146]:
# Calculando Prêmio Rentabilidade para TUMELERO
tina.loc[
    (tina.ID_UNN == 4) &  # ID_UNN para Tumelero
    (tina.job_simplified=='salesperson'), 
    'PREMIO_RENTABILIDADE'
    ] = tina.apply(
        lambda x: alvo_bonus_rentabilidade(
            4, # ID_UNN para Tumelero
            x['VENDAS_TOT'], x['rentability'], x['sales_segment'], x['Alvo1']
            ), axis=1).copy(deep=True)

In [147]:
# CHECANDO
tina[(tina['ID_UNN']==1) &
#(tina['job_simplified']=='salesperson') &
(tina['RECEBE_PREMIACAO']=='SIM')]

,ID_UNN,cod_loja,sigla_loja,MATRICULA,segment,job_simplified,E1_gross_sales,E2_gross_sales,E3_gross_sales,gross_sales,sales_segment,rentability,VENDAS_TOT,COMISSAO,NUMERO_DE_LOJAS_COM_VENDA,Alvo1,Minimo Garantido,FALTA,VENDAS_SOBRE_ALVO,ATINGIU_ALVO,RECEBE_PREMIACAO,PREMIO_RENTABILIDADE
0,1,1,ANH,1890,E3,salesperson,68514.788,7084.583,343005.411,418604.782,343005.411,-345.958834,418604.78,4186.0478,1.0,268704.0,1866.0,NAO,1.557866,SIM,SIM,0.000000
4,1,1,ANH,10522,E3,salesperson,40902.166,4571.030,482674.918,530005.184,482674.918,100.506720,530005.18,5300.0518,1.0,268704.0,1866.0,NAO,1.97245,SIM,SIM,444.581218
5,1,1,ANH,13170,E3,salesperson,88603.133,9693.355,282418.837,381787.635,282418.837,101.249834,381787.63,3817.8763,1.0,268704.0,1866.0,NAO,1.420848,SIM,SIM,552.976592
12,1,1,ANH,27242,E1,assistant,51384.227,7904.467,32788.917,92077.611,51384.227,-190.570906,92077.61,920.7761,1.0,NaN,NaN,NAO,None,50mil,SIM,NaN
13,1,1,ANH,27359,E1,assistant,43644.627,7356.856,49167.624,100169.107,43644.627,-352.253926,97310.11,973.1011,1.0,NaN,NaN,NAO,None,30mil,SIM,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
896,1,75,AMZ,26408,E3,salesperson,49240.626,23143.755,99861.307,174717.368,99861.307,105.207012,174717.37,1747.1737,1.0,160000.0,1283.7,NAO,1.091984,SIM,SIM,719.979027
897,1,75,AMZ,26754,E3,salesperson,64212.325,18251.171,126706.307,209169.803,126706.307,-119.771389,209169.80,2091.6980,1.0,160000.0,1283.7,NAO,1.307311,SIM,SIM,0.000000
898,1,75,AMZ,26903,E1,salesperson,42376.086,23114.679,98278.546,163769.311,42376.086,-236.546596,163769.31,1637.6931,1.0,160000.0,1283.7,NAO,1.023558,SIM,SIM,0.000000
900,1,75,AMZ,27457,E3,salesperson,50758.561,26477.283,148355.148,228828.072,148355.148,32.827304,228828.07,2288.2807,1.0,160000.0,1283.7,NAO,1.430175,SIM,SIM,0.000000


In [148]:
# Salva verba rentabilidade TELHANORTE -->  salva_verba(empresa,verba,numero_verba,arquivo)
salva_verba(
    dict_EMPRESAS[1], 
    tina[['MATRICULA','PREMIO_RENTABILIDADE']][tina['ID_UNN']==1].copy(deep=True),
    4119,
    verbas_telha['F_VERBA_VEN_REN_SEM']) # verbas_telha['F_VERBA_ASS_SEM']
salva_verba(
    dict_EMPRESAS[1], 
    tina[['MATRICULA','PREMIO_RENTABILIDADE']][
        (tina['ID_UNN']==1) &
        (tina['RECEBE_PREMIACAO']=='SIM')].copy(deep=True),
        4119,
        verbas_telha['F_VERBA_VEN_REN'])


In [149]:
# Salva verba rentabilidade TUMELERO -->  salva_verba(empresa,verba,numero_verba,arquivo)
salva_verba(
    dict_EMPRESAS[4], # ID_UNN TUMELERO
    tina[['MATRICULA','PREMIO_RENTABILIDADE']][tina['ID_UNN']==4].copy(deep=True),
    4119,
    verbas_tume['F_VERBA_VEN_REN_SEM']) # verbas_tume['F_VERBA_ASS_SEM']
salva_verba(
    dict_EMPRESAS[4],  # ID_UNN TUMELERO
    tina[['MATRICULA','PREMIO_RENTABILIDADE']][
        (tina['ID_UNN']==4) &  # ID_UNN TUMELERO
        (tina['RECEBE_PREMIACAO']=='SIM')].copy(deep=True),
        4119,
        verbas_tume['F_VERBA_VEN_REN']) # verbas_tume['F_VERBA_VEN_REN']


In [150]:
# Calculating bonus for SALESPERSON TELHANORTE E TUMELERO
# NÃO TEM DIFERENÇA ENTRE EMPRESAS
# Verba 4120 - Segmento vendedores - DECORAÇÃO
def alvo_bonus_decoracao(vendas_seg,vendas_tot,alvo):
# Modelo Vendedores Prêmio DECORAÇÃO vendas
#    T E S T A D O     E     C H E C A D O
# Modelo Vendedores Comissão Turbinada Decoração
# Como as safras 1, 2 e 3 tem os mesmos valores, mantive só um valor para todas as safras
# gat_min_decoracao = 0.0  # Gatilho minimo decoração
# ang_coef_decoracao = 0.02  # Coeficiente angular decoração = 2%
# lin_coef_decoracao = 0.0  # Coeficiente Linear decoração = 0.0%

    global ANG_COEF_DECORACAO 
    if vendas_tot>=alvo:
        bonus = vendas_seg*ANG_COEF_DECORACAO
    else:
        bonus=0.0
    return bonus


In [151]:
# Calculando Verba DECORAÇÃO para TELHANORTE E TUMELERO
# SEM DIFERENÇA ENTRE EMPRESAS
tina.loc[
    tina.job_simplified=='salesperson', 
    'PREMIO_DECORACAO'
    ] = tina.apply(
        lambda x: alvo_bonus_decoracao(
            x['E2_gross_sales'], x['VENDAS_TOT'], x['Alvo1']
            ), axis=1).copy(deep=True)


In [152]:
# Salva verba decoracao TELHANORTE -->  salva_verba(empresa,verba,numero_verba,arquivo)
salva_verba(
    dict_EMPRESAS[1], 
    tina[['MATRICULA','PREMIO_DECORACAO']][tina['ID_UNN']==1].copy(deep=True),
    4120,
    verbas_telha['F_VERBA_VEN_DEC_SEM']) # verbas_telha
salva_verba(
    dict_EMPRESAS[1], 
    tina[['MATRICULA','PREMIO_DECORACAO']][
        (tina['ID_UNN']==1) &
        (tina['RECEBE_PREMIACAO']=='SIM')].copy(deep=True),
        4120,
        verbas_telha['F_VERBA_VEN_DEC']) # verbas_telha

In [153]:
# Calculating bonus for SALESPERSON considering differences for TELHANORTE e TUMELERO 
def alvo_bonus_superacao(id_unn, vendas_tot): # id_unn 1=Telhanorte e 4=Tumelero
# Modelo Vendedores Prêmio superação vendas
# T E S T A D O    E    C H E C A D O 
#M1 = 100  # meta1 = 100%
#M2 = 150  # meta2 = 150%
#M3 = 200  # meta3 = 200%
#M4 = 250  # meta4 = 250%
#B1 = {1:300, 4:200}  # bonus1 -> ganha se bater a meta1; Telhanorte 1 = R$300,00; Tumelero 4= R$200,00
#B2 = {1:600, 4:400}  # bonus2 -> ganha se bater a meta2; Telhanorte 1 = R$600,00; Tumelero 4= R$400,00
#B3 = {1:900, 4:600}  # bonus3 -> ganha se bater a meta3; Telhanorte 1 = R$900,00; Tumelero 4= R$600,00
#B4 = {1:1200, 4:800}  # bonus4  -> ganha se bater a meta4; Telhanorte 1 = R$1200,00; Tumelero 4= R$800,00
    global M1, M2, M3, M4, B1, B2, B3, B4 # metas e respectivos bonus
    if vendas_tot>=M4:
        bonus = B4[id_unn]
    elif vendas_tot>=M3:
        bonus = B3[id_unn]
    elif vendas_tot>=M2:
        bonus= B2[id_unn]
    elif vendas_tot>=M1:
        bonus=B1[id_unn]
    else:
        bonus=0.0
    return bonus



In [154]:
# Calculando Prêmio Superação para TELHANORTE
tina.loc[
    (tina.ID_UNN == 1) &  # ID_UNN para TELHANORTE
    (tina.job_simplified=='salesperson'), 
   'PREMIO_SUPERACAO'
    ] = tina.apply(
        lambda x: alvo_bonus_superacao(
            1, # ID_UNN para TELHANORTE
            x['VENDAS_TOT']/x['Alvo1']*100.
            ), axis=1).copy(deep=True)


In [155]:
# Calculando Prêmio Superação para TUMELERO
tina.loc[
    (tina.ID_UNN == 4) &  # ID_UNN para Tumelero
    (tina.job_simplified=='salesperson'), 
   'PREMIO_SUPERACAO'
    ] = tina.apply(
        lambda x: alvo_bonus_superacao(
            4, # ID_UNN para Tumelero
            x['VENDAS_TOT']/x['Alvo1']*100.
            ), axis=1).copy(deep=True)

In [156]:
# Salva verba superação TELHANORTE -->  salva_verba(empresa,verba,numero_verba,arquivo)
salva_verba(
    dict_EMPRESAS[1], #
    tina[['MATRICULA','PREMIO_SUPERACAO']][tina['ID_UNN']==1].copy(deep=True),
    4121,
    verbas_telha['F_VERBA_VEN_SUP_SEM']) # verbas_telha['F_VERBA_ASS_SEM']
salva_verba(
    dict_EMPRESAS[1], 
    tina[['MATRICULA','PREMIO_SUPERACAO']][
        (tina['ID_UNN']==1) &
        (tina['RECEBE_PREMIACAO']=='SIM')].copy(deep=True),
        4121,
        verbas_telha['F_VERBA_VEN_SUP'])


In [157]:
# Salva verba superação TUMELERO -->  salva_verba(empresa,verba,numero_verba,arquivo)
salva_verba(
    dict_EMPRESAS[4], # TUMELERO 
    tina[['MATRICULA','PREMIO_SUPERACAO']][tina['ID_UNN']==4].copy(deep=True),
    4121,
    verbas_tume['F_VERBA_VEN_SUP_SEM']) # verbas TUMELERO['F_VERBA_ASS_SEM']
salva_verba(
    dict_EMPRESAS[4], # TUMELERO 
    tina[['MATRICULA','PREMIO_SUPERACAO']][
        (tina['ID_UNN']==4) & # TUMELERO 
        (tina['RECEBE_PREMIACAO']=='SIM')].copy(deep=True),
        4121,
        verbas_tume['F_VERBA_VEN_SUP']) # verbas TUMELERO


In [158]:
# para TELHANORTE E TUMELERO SEM DIFERENÇA ENTRE EMPRESAS
def alvo_bonus_areastecnicas(vendas_seg,vendas_tot,alvo):
# Modelo Vendedores Prêmio ÁREAS TÉCNICAS
#    T E S T A D O     E     C H E C A D O
# Modelo Vendedores Comissão  ÁREAS TÉCNICAS
# Como as safras 1, 2 e 3 tem os mesmos valores, mantive só um valor para todas as safras
# gat_min_areastecnicas = 0.0  # Gatilho minimo Áreas Técnicas = 0.0%
# ang_coef_areastecnicas = 0.005  # Coeficiente angular Áreas Técnicas = 0.5%
# lin_coef_areastecnicas = 0.0  # Coeficiente Linear decoração = 0.0%

    global ANG_COEF_AREASTECNICAS # gat_min_areastecnicas, ang_coef_areastecnicas, lin_coef_areastecnicas 
    if vendas_tot>=alvo:
        bonus = vendas_seg*ANG_COEF_AREASTECNICAS
    else:
        bonus=0.0
    return bonus




In [159]:
# Calculando Verba DECORAÇÃO para TELHANORTE E TUMELERO
# SEM DIFERENÇA ENTRE EMPRESAS
#tina.loc[
#    tina.job_simplified=='salesperson', 
#    'PREMIO_DECORACAO'
#    ] = tina.apply(
#        lambda x: alvo_bonus_decoracao(
#            x['E2_gross_sales'], x['VENDAS_TOT'], x['Alvo1']
#            ), axis=1).copy(deep=True)

tina.loc[
    tina.job_simplified=='salesperson', 
    'PREMIO_AREAS_TECNICAS'
    ] = tina.apply(
        lambda x: alvo_bonus_areastecnicas(
            x['E1_gross_sales'],x['VENDAS_TOT'],x['Alvo1']
            ), axis=1).copy(deep=True)



# TODO DELETAR SE OK
# Calculando Premiação Individual ÁREAS TÉCNICAS
#tina['AREAS_TEC'] = np.where(
#    tina['job_simplified']=='salesperson',
 #   tina.apply(
#    lambda x: alvo_bonus_areastecnicas(x['E1_gross_sales'],x['VENDAS_TOT'],x['Alvo1']), axis=1).copy(),np.nan
#    )

In [160]:
# Salva verba áreas técnicas TELHANORTE -->  salva_verba(empresa,verba,numero_verba,arquivo)
salva_verba(
    dict_EMPRESAS[1], # TELHANORTE
    tina[['MATRICULA','PREMIO_AREAS_TECNICAS']][tina['ID_UNN']==1].copy(deep=True), # TELHANORTE
    4122,
    verbas_telha['F_VERBA_VEN_TEC_SEM']) # verbas # TELHANORTE
salva_verba(
    dict_EMPRESAS[1], # TELHANORTE
    tina[['MATRICULA','PREMIO_AREAS_TECNICAS']][
        (tina['ID_UNN']==1) & # TELHANORTE
        (tina['RECEBE_PREMIACAO']=='SIM')].copy(deep=True),
        4122,
        verbas_telha['F_VERBA_VEN_TEC']) # verbas # TELHANORTE


In [161]:
# Salva verba áreas técnicas TUMELERO -->  salva_verba(empresa,verba,numero_verba,arquivo)
salva_verba(
    dict_EMPRESAS[4], # TUMELERO=4
    tina[['MATRICULA','PREMIO_AREAS_TECNICAS']][tina['ID_UNN']==4].copy(deep=True), # TUMELERO=4
    4122,
    verbas_tume['F_VERBA_VEN_TEC_SEM']) # verbas # TUMELERO
salva_verba(
    dict_EMPRESAS[4], # TUMELERO=4
    tina[['MATRICULA','PREMIO_AREAS_TECNICAS']][
        (tina['ID_UNN']==4) & # TUMELERO=4
        (tina['RECEBE_PREMIACAO']=='SIM')].copy(deep=True),
        4122,
        verbas_tume['F_VERBA_VEN_TEC']) # verbas TUMELERO


In [162]:
# Calculando Prêmio Assessor SOMENTE TELHANORTE tem assessores
def alvo_bonus_assessor(vendas_seg):
    global ASS_A1, ASS_A2, ASS_B1, ASS_B2 # ass_a1, ass_a2, ass_b1, ass_b2
    if vendas_seg >= ASS_A2:
        bonus = ASS_B2
    elif vendas_seg >= ASS_A1:
        bonus = ASS_B1
    else:
        bonus = 0.0
    return bonus
    

In [163]:
# Calculando prêmio dos assessores - somente TELHANORTE tem assessores
tina['PREMIO_ASSESSOR'] = np.where(
    tina['job_simplified']=='assistant',
    tina.apply(
    lambda x: alvo_bonus_assessor(x['sales_segment']), axis=1).copy(),np.nan
    )

    

In [164]:
# Salva verba assessores somente TELHANORTE -->  salva_verba(empresa,verba,numero_verba,arquivo)
salva_verba(
    dict_EMPRESAS[1], # TELHANORTE
    tina[['MATRICULA','PREMIO_ASSESSOR']][tina['ID_UNN']==1].copy(deep=True), # TELHANORTE
    4118, #VERBA PREMIAÇÃO ASSESSOR
    verbas_telha['F_VERBA_ASS_SEM']) # verbas # TELHANORTE
salva_verba(
    dict_EMPRESAS[1], # TELHANORTE
    tina[['MATRICULA','PREMIO_ASSESSOR']][
        (tina['ID_UNN']==1) & # TELHANORTE
        (tina['RECEBE_PREMIACAO']=='SIM')].copy(deep=True),
        4118, #VERBA PREMIAÇÃO ASSESSOR
        verbas_telha['F_VERBA_ASS']) # verbas # TELHANORTE


In [165]:
tina['DIFERENCA_VENDAS_ADP_totTINAsegm']=tina['VENDAS_TOT']-tina['gross_sales']

In [166]:
tina[tina['MATRICULA']==3343]

,ID_UNN,cod_loja,sigla_loja,MATRICULA,segment,job_simplified,E1_gross_sales,E2_gross_sales,E3_gross_sales,gross_sales,sales_segment,rentability,VENDAS_TOT,COMISSAO,NUMERO_DE_LOJAS_COM_VENDA,Alvo1,Minimo Garantido,FALTA,VENDAS_SOBRE_ALVO,ATINGIU_ALVO,RECEBE_PREMIACAO,PREMIO_RENTABILIDADE,PREMIO_DECORACAO,PREMIO_SUPERACAO,PREMIO_AREAS_TECNICAS,PREMIO_ASSESSOR,DIFERENCA_VENDAS_ADP_totTINAsegm
445,1,30,MOR,3343,E3,salesperson,51685.504,3622.766,337217.547,393523.817,337217.547,53.041764,393523.81,3935.2381,1.0,270000.0,1985.0,NAO,1.457496,SIM,SIM,0.0,72.45532,300.0,258.42752,NaN,-0.007


In [167]:
tina.columns

Index(['ID_UNN', 'cod_loja', 'sigla_loja', 'MATRICULA', 'segment',
       'job_simplified', 'E1_gross_sales', 'E2_gross_sales', 'E3_gross_sales',
       'gross_sales', 'sales_segment', 'rentability', 'VENDAS_TOT', 'COMISSAO',
       'NUMERO_DE_LOJAS_COM_VENDA', 'Alvo1', 'Minimo Garantido', 'FALTA',
       'VENDAS_SOBRE_ALVO', 'ATINGIU_ALVO', 'RECEBE_PREMIACAO',
       'PREMIO_RENTABILIDADE', 'PREMIO_DECORACAO', 'PREMIO_SUPERACAO',
       'PREMIO_AREAS_TECNICAS', 'PREMIO_ASSESSOR',
       'DIFERENCA_VENDAS_ADP_totTINAsegm'],
      dtype='object')

In [168]:
tina.to_excel(F_SAI_CHECAGEM)